In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:        
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor
import optuna
from itertools import combinations
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Raw_data=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')

In [ ]:
import kagglehub
path = kagglehub.dataset_download("senkin13/rdkit-2025-3-3-cp311")
print("Path to dataset files", path)

In [ ]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

In [ ]:
import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem

In [ ]:
df1=pd.read_csv('/kaggle/input/neurips-dataset/tg.csv')
df2=pd.read_csv('/kaggle/input/neurips-dataset/ffv.csv')
df3=pd.read_csv('/kaggle/input/neurips-dataset/tc.csv')
df4=pd.read_csv('/kaggle/input/neurips-dataset/density.csv')
df5=pd.read_csv('/kaggle/input/neurips-dataset/rg.csv')

In [ ]:
def GodMod(df, test,target):
    pred1 = np.zeros(len(test))
    
    y = df.iloc[:, 0]
    X = df.iloc[:, 1:]
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=69)
    xgb_params = {
        "objective": "reg:absoluteerror",
        "eval_metric": "mae",
        "n_estimators": 2000,
        "learning_rate": 0.02,
        "max_depth": 6,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "colsample_bylevel": 0.8,
        "reg_alpha": 0.1,
        "reg_lambda": 0.1,
        "random_state": 69,
        "n_jobs": -1,
        "verbosity": 0
    }
    if target == 1:
        xgb_params.update({
            "n_estimators": 3000,
            "learning_rate": 0.01,
            "max_depth": 8,
            "reg_alpha": 0.05,
            "reg_lambda": 0.05
        })
    model_xgb = xgb.XGBRegressor(**xgb_params)
    model_xgb.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=100,
                  verbose=False)

    pred1 = model_xgb.predict(test)
   # pred1 /= 50
    pred2 = np.zeros(len(test))
   
    y = df.iloc[:, 0]
    X = df.iloc[:, 1:]
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=69)
    lgb_params = {
        "objective": "mae",
        "metric": "mae",
        "n_estimators": 2000,
        "learning_rate": 0.02,
        "num_leaves": 63,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 1,
        "min_child_samples": 20,
        "random_state": 69,
        "n_jobs": -1,
        "verbosity": -1
    }
    if target == 1:
        lgb_params.update({
            "n_estimators": 3000,
            "learning_rate": 0.01,
            "num_leaves": 127,
            "feature_fraction": 0.7,
            "min_child_samples": 10
        })
    model_lgb = lgb.LGBMRegressor(**lgb_params)
    model_lgb.fit(X_train, y_train,
                 eval_set=[(X_val, y_val)],
                 callbacks=[lgb.early_stopping(100, verbose=False)])
    pred2=model_lgb.predict(test)
    #pred2/=50
    pred3 = np.zeros(len(test))
   
    y = df.iloc[:, 0]
    X = df.iloc[:, 1:]
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=69)
    catboost_params = {
        "iterations": 3000,
        "learning_rate": 0.05,
        "depth": 7,
        "l2_leaf_reg": 3,
        "random_seed": 69,
        "loss_function": "MAE",
        "task_type": "CPU",
        "bootstrap_type": "Bernoulli",
        "subsample": 0.8,
        "verbose": False,
        "od_type": "Iter",
        "od_wait": 100
    }
    
        # Special settings for Density target
    if target == 2:
        catboost_params.update({
            "depth": 9,
            "learning_rate": 0.03,
            "l2_leaf_reg": 5
        })
    
    cat_train = Pool(
        X_train, y_train,
        cat_features=[col for col in X.columns if X[col].dtype == 'object']
    )
    cat_val = Pool(
        X_val, y_val,
        cat_features=[col for col in X.columns if X[col].dtype == 'object']
    )
    model_cat = CatBoostRegressor(**catboost_params)
    model_cat.fit(cat_train,
                 eval_set=cat_val,
                 early_stopping_rounds=100,
                 verbose=False)
    pred3=model_cat.predict(test)
    #pred3/=50
    pred=(pred1+pred2+pred3)/3
    
    return pred

In [ ]:
#extra tree
p1={'n_estimators': 2116, 'max_depth': 5, 'learning_rate': 0.08282502750192564, 'subsample': 0.6793910752113601, 'colsample_bytree': 0.7442014260176102, 'gamma': 4.755722698540227, 'reg_alpha': 0.06917547165803263, 'reg_lambda': 0.7353510890753056}
p2={'n_estimators': 1855, 'max_depth': 5, 'learning_rate': 0.06145325680863284, 'subsample': 0.8699297228633653, 'colsample_bytree': 0.9933170293118256, 'gamma': 0.00014825488273781091, 'reg_alpha': 0.0502587910984648, 'reg_lambda': 0.6493903850371053}
p3={'n_estimators': 700, 'max_depth': 7, 'learning_rate': 0.23893917169396547, 'subsample': 0.8046315493363868, 'colsample_bytree': 0.706559663958085, 'gamma': 0.0007392391224947704, 'reg_alpha': 0.07109604199415018, 'reg_lambda': 0.35302009921885363}
p4={'n_estimators': 910, 'max_depth': 5, 'learning_rate': 0.06401768100530354, 'subsample': 0.8143668727193376, 'colsample_bytree': 0.9240712642977564, 'gamma': 0.00010692540275645759, 'reg_alpha': 0.058782776860916625, 'reg_lambda': 0.02446757762273763}
p5={'n_estimators': 1411, 'max_depth': 13, 'learning_rate': 0.10131862608582638, 'subsample': 0.6424546915143092, 'colsample_bytree': 0.6809981271570222, 'gamma': 0.8257122531304489, 'reg_alpha': 0.675715562941502, 'reg_lambda': 0.9412687084570486}

In [ ]:
# import optuna
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error
# import numpy as np

# # Your 5 datasets
# datasets = [df1, df2, df3, df4, df5]

# # Store best models, test predictions, and best params
# best_models = []
# test_preds = []
# best_params_list = []

# # Define objective function
# def objective(trial, X, y):
#     model = ExtraTreesRegressor(
#         n_estimators=trial.suggest_int('n_estimators', 100, 1000),
#         max_depth=trial.suggest_int('max_depth', 3, 20),
#         min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
#         min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
#         max_features=trial.suggest_float('max_features', 0.1, 1.0),
#         bootstrap=trial.suggest_categorical('bootstrap', [True, False]),
#         random_state=42,
#         n_jobs=-1
#     )
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#     model.fit(X_train, y_train)
#     preds = model.predict(X_val)
#     return mean_absolute_error(y_val, preds)

# # Loop over datasets
# for idx, df in enumerate(datasets):
#     print(f"\n🔍 Tuning for Target {idx+1}...")

#     y = df.iloc[:, 0]
#     X = df.iloc[:, 1:]

#     # Run Optuna
#     study = optuna.create_study(direction='minimize')
#     study.optimize(lambda trial: objective(trial, X, y), n_trials=50, timeout=300)

#     # Get best parameters and model
#     best_params = study.best_params
#     best_params_list.append(best_params)
    
#     print(f"✅ Best parameters for df{idx+1}:\n{best_params}\n")

#     model = ExtraTreesRegressor(**best_params, random_state=42, n_jobs=-1)
#     model.fit(X, y)
#     best_models.append(model)

#     # Predict (you can customize testX as needed)
#   #  test_pred = model.predict(testX)  # Replace testX with your actual test data
# #    test_preds.append(test_pred)

# # Final summary of best parameters
# print("\n🎯 All Best Parameters by Target:\n")
# for i, params in enumerate(best_params_list, 1):
#     print(f"Target {i}: {params}")


**prediction/submission**

In [ ]:
test=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv', dtype=str)

In [ ]:
prodata=test

In [ ]:
useless_cols = [   
    
    'MaxPartialCharge', 
    # Nan data
    'BCUT2D_MWHI',
    'BCUT2D_MWLOW',
    'BCUT2D_CHGHI',
    'BCUT2D_CHGLO',
    'BCUT2D_LOGPHI',
    'BCUT2D_LOGPLOW',
    'BCUT2D_MRHI',
    'BCUT2D_MRLOW',

    # Constant data
    'NumRadicalElectrons',
    'SMR_VSA8',
    'SlogP_VSA9',
    'fr_barbitur',
    'fr_benzodiazepine',
    'fr_dihydropyridine',
    'fr_epoxide',
    'fr_isothiocyan',
    'fr_lactam',
    'fr_nitroso',
    'fr_prisulfonamd',
    'fr_thiocyan',

    # High correlated data >0.95
    'MaxEStateIndex',
    'HeavyAtomMolWt',
    'ExactMolWt',
    'NumValenceElectrons',
    'Chi0',
    'Chi0n',
    'Chi0v',
    'Chi1',
    'Chi1n',
    'Chi1v',
    'Chi2n',
    'Kappa1',
    'LabuteASA',
    'HeavyAtomCount',
    'MolMR',
    'Chi3n',
    'BertzCT',
    'Chi2v',
    'Chi4n',
    'HallKierAlpha',
    'Chi3v',
    'Chi4v',
    'MinAbsPartialCharge',
    'MinPartialCharge',
    'MaxAbsPartialCharge',
    'FpDensityMorgan2',
    'FpDensityMorgan3',
    'Phi',
    'Kappa3',
    'fr_nitrile',
    'SlogP_VSA6',
    'NumAromaticCarbocycles',
    'NumAromaticRings',
    'fr_benzene',
    'VSA_EState6',
    'NOCount',
    'fr_C_O',
    'fr_C_O_noCOO',
    'NumHDonors',
    'fr_amide',
    'fr_Nhpyrrole',
    'fr_phenol',
    'fr_phenol_noOrthoHbond',
    'fr_COO2',
    'fr_halogen',
    'fr_diazo',
    'fr_nitro_arom',
    'fr_phos_ester'
]

In [ ]:
def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
    descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['SMILES']:
         compute_graph_features(smile, graph_feats)
        
    result = pd.concat(
        [
            pd.DataFrame(descriptors, columns=desc_names),
            pd.DataFrame(graph_feats)
        ],
        axis=1
    )

    result = result.replace([-np.inf, np.inf], np.nan)
    return result

In [ ]:
def compute_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(desc_names)
    return [desc[1](mol) for desc in Descriptors.descList if desc[0] not in useless_cols]

def compute_graph_features(smiles, graph_feats):
    mol = Chem.MolFromSmiles(smiles)
    adj = rdmolops.GetAdjacencyMatrix(mol)
    G = nx.from_numpy_array(adj)

    graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
    graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
    graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

test = pd.concat([test, preprocessing(test)], axis=1)
test['Ipc']=np.log10(test['Ipc'])

test=test.drop(['id','SMILES'],axis=1)

In [ ]:
test.head()

In [ ]:
prodata=test

In [ ]:
pred1=GodMod(df1,prodata,1)
pred2=GodMod(df2,prodata,0)
pred3=GodMod(df3,prodata,0)
pred4=GodMod(df4,prodata,2)
pred5=GodMod(df5,prodata,0)

In [ ]:
for i in range (len(pred4)):
    if(pred4[i]<0):
        pred4[i]=pred4[i]*(-1)

for i in range (len(pred5)):
    if(pred5[i]<0):
        pred5[i]=pred5[i]*(-1)


for i in range (len(pred3)):
    if(pred3[i]<0):
        pred3[i]=pred3[i]*(-1)

In [ ]:
ss=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')

In [ ]:
ss['Tg']=pred1
ss['FFV']=pred2
ss['Tc']=pred3
ss['Density']=pred4
ss['Rg']=pred5

In [ ]:
submission=ss

In [ ]:
ss.head()

In [ ]:
submission.to_csv('submission.csv',index =False)